In [1]:
from langgraph.graph import StateGraph , START , END
from langchain_google_genai import  ChatGoogleGenerativeAI
from typing import TypedDict
from dotenv import load_dotenv

/home/powder-03/langgraph/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
load_dotenv()
model = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0.7)

In [3]:
class BlogState(TypedDict):
    """State for the blog workflow."""
    title: str
    outline: str
    content: str

In [4]:
def create_outline(state: BlogState) -> BlogState:
    
    # fetch title
    title = state['title']
    
    # call llm gen outline
    
    prompt = f'Generate a detailed outline for a blog on the topic - {title}'
    outline = model.invoke(prompt).content
    
    # update state
    state['outline'] = outline
    
    return state

In [5]:
def create_blog(state : BlogState) -> BlogState:
    
    title = state['title']
    outline = state['outline']
    
    prompt = f'Write a detailed blog on the title - {title} using the following outline \n {outline}'
    
    content = model.invoke(prompt).content
    
    state['content'] = content
    
    return state

In [9]:
graph = StateGraph(BlogState)

#node 
graph.add_node('create_outline', create_outline)
graph.add_node('create_blog', create_blog)

# edges

graph.add_edge (START , 'create_outline')
graph.add_edge('create_outline' , 'create_blog')
graph.add_edge('create_blog' , END)

workflow = graph.compile()

In [10]:
initial_state = {'title':'Rise of AI in India'}

final_state = workflow.invoke(initial_state)

print(final_state)

{'title': 'Rise of AI in India', 'outline': 'Here\'s a detailed outline for a blog post on the "Rise of AI in India," designed to be engaging, informative, and well-structured.\n\n---\n\n## Blog Title Options:\n\n*   **India\'s AI Ascent: Unpacking the Revolution Transforming the Nation**\n*   **The AI Wave: How India is Embracing and Shaping the Future of Technology**\n*   **From Silicon Valley to Bengaluru: The Unstoppable Rise of AI in India**\n*   **Beyond the Hype: A Deep Dive into India\'s AI Journey**\n\n---\n\n## Detailed Blog Outline: India\'s AI Ascent\n\n### I. Introduction (Approx. 150-200 words)\n\n*   **A. Catchy Hook:** Start with a compelling statistic, a thought-provoking question, or a vivid image related to AI\'s global impact.\n    *   *Example:* "From self-driving cars to personalized healthcare, Artificial Intelligence is reshaping our world at an unprecedented pace. But while global tech giants often dominate the narrative, a quiet yet powerful revolution is brew